In [1]:
from scipy import signal
import numpy as np
from scipy.ndimage import gaussian_filter
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.utils import shuffle


import sys
sys.path.append('../')
from Utils.constants import TRAIN_FILES, TEST_FILES
from Utils.perturbations import RBPIndividual, zeroPerturb, noisePerturb, blurPerturb
from Utils.data import *

In [2]:
X_train, y_train, X_test, y_test, nb_classes = loadUCRDataID(0)

Loading train / test dataset :  ../Data/Coffee_TRAIN ../Data/Coffee_TEST


In [3]:
#The first multisample background generation approac. 
#All spectral matrixs are superpositioned and treated as a single sample.

def multisamplebackgroundIdentification(global_signals):
    test_sig = global_signals[0].flatten()
    f, t, Zxx = signal.stft(test_sig,1,nperseg=40)
    for i in range(1,len(global_signals)):
        test_sig = global_signals[i].flatten()
        f0, t0, Zxx0 = signal.stft(test_sig,1,nperseg=40, noverlap = 20)
        assert Zxx.shape == Zxx0.shape, f"Shape mismatch at signal {i}"
        Zxx = Zxx + Zxx0
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud') 
    plt.title('STFT Magnitude Spectrum')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [s]')
    plt.colorbar(label='Magnitude')
    plt.ylim(np.min(f), 0.1)
    plt.show()
    frequency_composition_abs = np.abs(Zxx)
    measures = []
    for freq,freq_composition in zip(f,frequency_composition_abs):
        measures.append(np.mean(freq_composition)/np.std(freq_composition))
    max_value = max(measures)
    selected_frequency = measures.index(max_value)
    weights = 1-(measures/sum(measures))
    dummymatrix = np.zeros((len(f),len(t)))
    dummymatrix[selected_frequency,:] = 1  
    #Option to admit information from other frequency bands
    """dummymatrix = np.ones((len(f),len(t)))
    for i in range(0,len(weights)):
        dummymatrix[i,:] = dummymatrix[i,:] * weights[i]"""
    
    background_frequency = Zxx * dummymatrix
    _, xrec = signal.istft(background_frequency, 1)
    return xrec,f,t,Zxx



In [4]:
#rec,f,t,Zxx = multisamplebackgroundIdentification(X_train)

In [5]:
#The second background identification approach which combines the most significant frequency component 
#of every sample into a spectral matrix and identify the background in this matrix


#Identify backgrounds in a single TS
def backgroundfreqIdentification(original_signal):
    f, t, Zxx = signal.stft(original_signal,fs = 1,nperseg=40, noverlap = 20)
    frequency_composition_abs = np.abs(Zxx)
    measures = []
    for freq,freq_composition in zip(f,frequency_composition_abs):
        measures.append(np.mean(freq_composition)/np.std(freq_composition))
    max_value = max(measures)
    selected_frequency = measures.index(max_value)
    Zxx_candidate = np.zeros_like(Zxx, dtype=complex)
    # insert the selected component into selected frequency
    Zxx_candidate[selected_frequency, :] = Zxx[selected_frequency, :]
    return Zxx_candidate



#Background Identification
def candidatebackgroundIdentification(global_signals):
    #Set up template matrix
    test_sig = global_signals[0].flatten()
    f_, t_, Zxx_ = signal.stft(test_sig,1,nperseg=40)
    Zxx = np.zeros_like(Zxx_, dtype=complex)

    #For every sample, return a spectral matrix representing the background 
    for i in range(1,len(global_signals)):
        test_sig = global_signals[i].flatten()
        Zxx_candidate = backgroundfreqIdentification(test_sig)
        assert Zxx.shape == Zxx_candidate.shape, f"Shape mismatch at signal {i}"
        #Add all backgrounds together
        Zxx = Zxx + Zxx_candidate
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(t_, f_, np.abs(Zxx), shading='gouraud') 
    plt.title('STFT Magnitude Spectrum')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [s]')
    plt.colorbar(label='Magnitude')
    
    plt.ylim(np.min(f_), 0.1)
    
    plt.show()
    frequency_composition_abs = np.abs(Zxx)
    measures = []
    for freq,freq_composition in zip(f_,frequency_composition_abs):
        measures.append(np.mean(freq_composition)/np.std(freq_composition))
    max_value = max(measures)
    selected_frequency = measures.index(max_value)
    weights = 1-(measures/sum(measures))
    dummymatrix = np.zeros((len(f_),len(t_)))
    dummymatrix[selected_frequency,:] = 1  
    
    background_frequency = Zxx * dummymatrix
    _, xrec = signal.istft(background_frequency, 1)
    return xrec,f_,t_,Zxx


#rec,f,t,Zxx = candidatebackgroundIdentification(X_train)

In [6]:
#The original approach

def backgroundIdentification(original_signal):
    f, t, Zxx = signal.stft(original_signal,fs = 1,nperseg=40, noverlap = 20)
    print(Zxx.shape)
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud') 
    plt.title('STFT Magnitude Spectrum')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [s]')
    plt.colorbar(label='Magnitude')
    plt.ylim(np.min(f), 0.1)
    plt.show()
    frequency_composition_abs = np.abs(Zxx)
    measures = []
    for freq,freq_composition in zip(f,frequency_composition_abs):
        measures.append(np.mean(freq_composition)/np.std(freq_composition))
    max_value = max(measures)
    selected_frequency = measures.index(max_value)
    weights = 1-(measures/sum(measures))
    dummymatrix = np.zeros((len(f),len(t)))
    dummymatrix[selected_frequency,:] = 1  
    
    background_frequency = Zxx * dummymatrix
    _, xrec = signal.istft(background_frequency, 1)
    return xrec

print(len(X_train[0]))
test_sig = X_train[0].flatten()
#print(test_sig)
#f, t, Zxx = signal.stft(test_sig,fs = 1,nperseg=40, noverlap = 20)
#rec = backgroundIdentification(test_sig)



286


In [7]:
def perturb(perturbation_strategy, ts, index0, index1, global_ts = []):
    if perturbation_strategy == 'RBP':
        return RBPIndividual(ts, index0, index1)
    if perturbation_strategy == 'zero':
        return zeroPerturb(ts, index0, index1)
    if perturbation_strategy == 'noise':
        return noisePerturb(ts, index0, index1)
    if perturbation_strategy == 'blur':
        return blurPerturb(ts, index0, index1)
    if perturbation_strategy == 'RBP1':
        return RBPIndividualNew1(global_ts, ts, index0, index1)    
    if perturbation_strategy == 'RBP2':
        return RBPIndividualNew2(global_ts, ts, index0, index1)    